Copyright 2019 Almintas Povilaitis

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

<table class="tfo-notebook-buttons" align="left">
<td>
<a target="_blank"  href="https://colab.research.google.com/github/mlai-demo/TextExplore/blob/master/RePlutarch_ExplorePub.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/mlai-demo/TextExplore/blob/master/RePlutarch_ExplorePub.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

## Pre-processing

### Download the libraries and dataset

Check the current directory and upload the text file:

In [ ]:
import os
fpath = os.getcwd(); fpath

In [ ]:
# use for file uploading into Google Colab Files folder
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Click the Files tab on the left - the uploaded file(s) will be there

In [ ]:
book = open(fpath + '/Plutarch.txt', 'rb').read().decode(encoding='utf-8')
words = [w for w in book.split(' ') if w.strip() != '' or w == '\n']
print("The text is {} words long".format(len(words)))

### Explore the text
Tokenize, convert to lower case, remove stop words and some punctuation, then save the new text for future reference:

In [ ]:
# Check the English stop words in case need to enhance the list
import nltk
nltk.download('stopwords') #need if using Google Colab
from nltk.corpus import stopwords
print (stopwords.words('english'))

In [ ]:
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt') #need if using Google Colab
import matplotlib.pyplot as plt
%matplotlib inline

with open(fpath + '/Plutarch.txt') as f, open(fpath + '/Plutarch_tokens.txt', 'w') as out_f:
    text = f.read()
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = stopwords.words('english')
    new_stop_words = ['one','came', 'come', 'upon', 'made','though', 'indeed', 'left', 'yet', 'without'
                 'thus','therefore', 'another', 'much', 'many','said', 'either', 'two', 'upon', 'would',
                     'around', 'without', 'when', 'also', 'could', 'sent', 'notwithstanding', 'hence', 'thus']
    stop_words.extend(new_stop_words)
    words = [w for w in words if not w in stop_words]
    new_text = ' '.join(words)
    plt.figure(figsize=(16, 7))
    fd = nltk.FreqDist(words)
    fd.plot(40,title = "40 Most Frequent Words", cumulative=False)
    #print(new_text[:500]) uncomment if you want to print some text
    out_f.write(new_text)

In [ ]:
fd.most_common(40)

In [ ]:
bigrams = nltk.bigrams(words)
freq_bigrams = nltk.FreqDist(bigrams)

plt.figure(figsize=(13, 7))
freq_bigrams.plot(20)

In [ ]:
freq_bigrams.most_common(20)

In [ ]:
trigrams = nltk.trigrams(words)
freq_trigrams = nltk.FreqDist(trigrams)

plt.figure(figsize=(13, 7))
freq_trigrams.plot(20)

In [ ]:
freq_trigrams.most_common(20)

In [ ]:
unique = set(words)
print("The tokenized text is {} words long, has {} unique words and {} letters on average".format
      (len(words), len(unique), round(sum(len(word) for word in words)/len(words),2)))

In [ ]:
from nltk.stem.porter import PorterStemmer
with open(fpath + '/Plutarch_tokens.txt') as f, open(fpath + '/Plutarch_stem.txt', 'w') as out_f:
    text = f.read()
    tokens = word_tokenize(text)
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in tokens]
    print(stemmed[:100])
    new_stem_text = ' '.join(stemmed)
    fd_stemmed = nltk.FreqDist(stemmed)
    out_f.write(new_stem_text)

In [ ]:
unique_stemmed = set(stemmed)
print("The stemmed text is {} words long, has {} unique words and {} letters on average".format
      (len(stemmed), len(unique_stemmed), round(sum(len(word) for word in stemmed)/len(stemmed),2)))

In [ ]:
nltk.download('wordnet') #need if using Google Colab
from nltk.stem import WordNetLemmatizer

with open(fpath + '/Plutarch_tokens.txt') as f, open(fpath + '/Plutarch_lemma.txt', 'w') as out_f:
    text = f.read()
    tokens = word_tokenize(text)
    lemma = WordNetLemmatizer()
    lemmed = [lemma.lemmatize(word) for word in tokens]
    print(lemmed[:100])
    new_lem_text = ' '.join(lemmed)
    out_f.write(new_lem_text)

In [ ]:
unique_lemmed = set(lemmed)
print("The lemmatized text is {} words long, has {} unique words and {} letters on average".format
      (len(lemmed), len(unique_lemmed), round(sum(len(word) for word in lemmed)/len(lemmed),2)))

Unique words will help us determining the dictionary size in embeddings

Let's convert the into a word cloud! The wordcloud library has its own list of stop words, so we could inspect it - but here we'll be using the previously defined stop words (NLTK's stop words plus some additional ones indicated above); technically, the stop words are not even needed in this  case because the text files being used had already been cleaned.

In [ ]:
#from wordcloud import STOPWORDS
#print(STOPWORDS)

# In case adding stop words is needed, it is as simple as:
#stopwords = set(STOPWORDS)
#stopwords.add("additional_stopword")

In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt  
from os import path

d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()

text_standard = open(fpath + '/Plutarch_tokens.txt', 'rt').read()

wc = WordCloud(stopwords=stop_words,
                      max_font_size=200,
                      width=2500,
                      height=2000,
                      max_words=4000,
                      random_state=44,
                     ).generate(text_standard)

wc.to_file(path.join(d, "wc.png")) #comment out if no need to store the png

fig = plt.figure().set_size_inches(18, 18)
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Plutarch's Lives", fontsize= 20)
plt.show()

Now let's use the stemmed text and a more artistic way of creating the word cloud

In [ ]:
import numpy as np
from PIL import Image

roman_mask = np.array(Image.open(path.join(d, 'roman7.png')))
text_stem = open(fpath + '/Plutarch_stem.txt', 'rt').read()

wc = WordCloud(stopwords=stop_words, 
               max_font_size=200,
               width=5000,
               height=4000,
               background_color="white", 
               max_words=4000, 
               mask=roman_mask,
               contour_width=3, 
               contour_color='steelblue',
               random_state = 64).generate(text_stem)

wc.to_file(path.join(d, "wc_stemmed.png")) #comment out if no need to store the png

fig = plt.figure().set_size_inches(18, 18)
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.title("Plutarch's Lives (Stemmed)", fontsize= 20)
plt.show()

In [ ]:
roman_mask = np.array(Image.open(path.join(d, 'roman8.png')))
text_lemma = open(fpath + '/Plutarch_lemma.txt', 'rt').read()

wc = WordCloud(stopwords=stop_words, 
               max_font_size=200,
               width=5000,
               height=4000,
               background_color="black", 
               max_words=4000, 
               mask=roman_mask,
               contour_width=3, 
               contour_color='red',
               random_state = 64).generate(text_lemma)

wc.to_file(path.join(d, "wc_lemmed.png")) #comment out if no need to store the png

fig = plt.figure().set_size_inches(18, 18)
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.title("Plutarch's Lives (Lemmatized)", fontsize= 20)
plt.show()